# Value Candidates with FMP Stock Screen 
#### Source Article: [Building an Investment Model using Financial Ratios in Python](https://codingandfun.com/building-an-investing-model-using-financial-ratios-and-python/#comments) 
---
## Basic Criteria:

* D/E <= 0.50 low debt 
* Current Ratio > 1.5 more than enough cash to cover current liabilities
* ROE stable growth for the past 10-years
* FCF stable growth last 10 years (cash-rich)
* Operating Margin - stable operating margins over the last 10 years (strong competitive position)
* Companies that grow without issuing additional shares

In [28]:
import numpy as np
import pandas as pd
import datetime as datetime
from datetime import datetime
import requests
import os
import json
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")

In [29]:
load_dotenv()

True

In [30]:
# Read in API key from env file
FMP_API_KEY = os.getenv('FMP_API_KEY')

# Create Basic Stock Screen

In [31]:
companies = []

market_cap = '1000000000' 
volume = '10000'
beta = 1
dividend = 1
limit = 1000
exchange = 'NYSE,NASDAQ'

url = (f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan={market_cap}&betaMoreThan={beta}&volumeMoreThan={volume}&dividendMoreThan={dividend}&exchange={exchange}&limit={limit}&apikey={FMP_API_KEY}')

#get companies based on criteria defined about
screen = requests.get(url).json()
screen

[{'symbol': 'TSM',
  'companyName': 'Taiwan Semiconductor Manufacturing Company Limited',
  'marketCap': 624663265280,
  'sector': 'Technology',
  'industry': 'Semiconductors',
  'beta': 1,
  'price': 112.657,
  'lastAnnualDividend': 1.725109,
  'volume': 8007467,
  'exchange': 'New York Stock Exchange',
  'exchangeShortName': 'NYSE',
  'country': 'TW',
  'isEtf': False,
  'isActivelyTrading': True},
 {'symbol': 'VTI',
  'companyName': 'Vanguard Total Stock Market Index Fund ETF Shares',
  'marketCap': 477434249216,
  'sector': '',
  'industry': 'Closed-End Fund - Equity',
  'beta': 1.043525,
  'price': 224.355,
  'lastAnnualDividend': 1.4540000000000002,
  'volume': 3418839,
  'exchange': 'New York Stock Exchange Arca',
  'exchangeShortName': 'NYSE',
  'country': 'US',
  'isEtf': True,
  'isActivelyTrading': True},
 {'symbol': 'JPM',
  'companyName': 'JPMorgan Chase & Co.',
  'marketCap': 472637276160,
  'sector': 'Financial Services',
  'industry': 'Banks—Diversified',
  'beta': 1.18

# Pull Financial Ratios

In [33]:
# add selected companies to a list
for item in screen:
	companies.append(item['symbol'])
	
# print(companies)
value_ratios = {}

# get the financial ratios
count = 0

for company in companies:

	try:
		if count <100:
			count = count + 1
	
			fin_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={FMP_API_KEY}').json()

			value_ratios[company] = {}

			value_ratios[company]['ROE'] = fin_ratios[0]['returnOnEquity']
			value_ratios[company]['ROA'] = fin_ratios[0]['returnOnAssets']
			value_ratios[company]['Debt_Eq_Ratio'] = fin_ratios[0]['debtEquityRatio']
			value_ratios[company]['Current_Ratio'] = fin_ratios[0]['currentRatio']
			value_ratios[company]['PB'] = fin_ratios[0]['priceToBookRatio']
			value_ratios[company]['PS'] = fin_ratios[0]['priceToSalesRatio']
			value_ratios[company]['PE'] = fin_ratios[0]['priceEarningsRatio']
			value_ratios[company]['Dividend_Yield'] = fin_ratios[0]['dividendYield']
			value_ratios[company]['Operating_Margin'] = fin_ratios[0]['operatingProfitMargin']
			
			#more financials on growth
			growth_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/financial-growth/{company}?apikey={FMP_API_KEY}').json()
			
			value_ratios[company]['Revenue_Growth'] = growth_ratios[0]['revenueGrowth']
			value_ratios[company]['FCF_Growth'] = growth_ratios[0]['freeCashFlowGrowth']
			value_ratios[company]['Net_Income_Growth'] = growth_ratios[0]['netIncomeGrowth']
			value_ratios[company]['EPS_Growth'] = growth_ratios[0]['epsgrowth']
			value_ratios[company]['Avg_Shares_Dil_Growth'] = growth_ratios[0]['weightedAverageSharesDilutedGrowth']
		
								
	except:
		pass
	
#print(value_ratios)

In [34]:
df = pd.DataFrame.from_dict(value_ratios, orient='index')
df.T

,TSM,JPM,HD,MA,XOM,ACN,CVX,MS,TXN,UPS,...,LYB,WMB,FAST,AMP,LEN,SWK,SWKS,MPLX,PSX,FITB
ROE,0.279990,0.104280,3.899970,1.003129,-0.136721,0.291890,-0.041763,0.108036,0.609013,NaN,...,0.178146,0.013577,0.314320,0.261462,0.136194,0.111559,0.195668,-0.055312,-0.209387,0.061745
ROA,0.187591,0.008603,0.182287,0.190894,-0.067438,0.137757,-0.023116,0.009854,0.289132,0.021520,...,0.040110,0.004483,0.216687,0.009247,0.082346,0.052354,0.159555,-0.019773,-0.072641,0.006972
Debt_Eq_Ratio,0.492031,11.121076,20.394665,4.239712,1.027356,1.118877,0.806654,9.949922,1.106346,NaN,...,3.439343,2.221491,0.450571,27.273905,0.653932,1.130231,0.226334,1.797419,1.748736,7.856389
Current_Ratio,1.769721,11.898671,1.229258,1.613320,0.796498,1.401748,1.175585,NaN,4.284100,1.188058,...,2.114504,0.616214,4.079647,2.251761,12.902842,1.324178,5.168599,0.726270,1.394831,0.000839
PB,9.751356,1.418505,84.738245,49.539399,1.166614,8.217384,1.189827,1.168653,16.610481,NaN,...,3.593507,1.767219,9.557687,4.148289,1.332245,2.418532,5.717152,1.865915,1.568268,0.894688
PS,13.467853,3.314832,2.116051,20.691870,1.072252,3.244011,1.671635,2.627553,10.552555,1.586945,...,1.032099,3.338690,4.625763,2.035291,1.072212,1.840298,7.094604,2.945502,0.464252,2.858722
PE,34.827524,13.602863,21.727924,49.384854,-8.532814,28.152301,-28.490169,10.817270,27.274440,100.000000,...,20.171719,130.158342,30.407485,15.865717,9.781938,21.679361,29.218660,-33.734182,-7.489811,14.489936
Dividend_Yield,0.014376,0.032024,0.023076,0.005069,0.077634,0.014171,0.061113,0.023027,0.022451,0.025123,...,0.049051,0.075316,0.030754,0.020421,0.008089,0.016846,0.012895,0.123762,0.052902,0.041495
Operating_Margin,0.423213,0.979288,0.138354,0.533495,-0.164906,0.146945,-0.064538,0.087919,0.422931,0.021789,...,0.077145,0.285270,0.201937,0.739087,0.138544,0.129230,0.269869,0.406136,-0.023266,0.000000
Revenue_Growth,0.251638,0.033868,0.198548,-0.093704,-0.301307,0.025732,-0.355217,0.162950,0.005423,0.142171,...,-0.200824,-0.058773,0.058796,-0.072304,0.010301,0.006398,-0.006249,-0.043942,-0.402300,-0.002757


# Normalize dataframe values 

In [35]:
# calculate mean of dataframe columns to enable comparison across ratios
ratios_mean = []

for item in df.columns:

	ratios_mean.append(df[item].mean())

# divide each value in dataframe by mean to normalize values
df = df / ratios_mean

# Create Ranks

In [36]:
# Define rank values
Revenue_Growth = 1.25
ROE = 1.20
FCF_Growth = 1.20
Current_Ratio = 1.10
Operating_Margin = 1.05
DE = -1.20
PB = -1.15
Avg_Num_Shares = -1.10

# Add rank column to dataframe
df['rank'] = df['Revenue_Growth'] * Revenue_Growth + df['Operating_Margin'] * Operating_Margin + df['ROE'] * ROE  + df['FCF_Growth'] * FCF_Growth + df['Debt_Eq_Ratio'] * DE + df['Current_Ratio'] * Current_Ratio + df['PB'] * PB + df['Avg_Shares_Dil_Growth'] * Avg_Num_Shares
df = df.sort_values(by=['rank'],ascending=False)
df

,ROE,ROA,Debt_Eq_Ratio,Current_Ratio,PB,PS,PE,Dividend_Yield,Operating_Margin,Revenue_Growth,FCF_Growth,Net_Income_Growth,EPS_Growth,Avg_Shares_Dil_Growth,rank
JCI,0.133136,0.314967,0.273643,0.495167,0.242666,0.380852,1.723195,0.796930,0.449614,1.963245,-1.195283,-1.478237,-1.433313,-3056.937208,3364.219751
AMP,1.022403,0.188399,6.207589,0.914801,0.588529,0.545712,0.544212,0.653010,4.247434,2.060735,-0.682685,-0.315419,-0.312595,-2116.626213,2328.612587
C,0.215771,0.099581,2.341831,0.000000,0.085560,0.435706,0.374885,1.417525,0.859114,-0.004604,-0.213638,-0.716168,-0.605965,-1762.421281,1936.653692
QCOM,3.344720,2.975197,1.105500,0.867567,3.336304,1.628369,0.943036,0.644891,1.527630,-5.763385,0.205056,0.307916,0.451075,-1288.659692,1411.976147
LYB,0.696608,0.817155,0.782800,0.859039,0.509820,0.276731,0.691913,1.568533,0.443341,5.723661,0.235358,-0.967956,-0.958224,-1191.841825,1319.183740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USB-PP,0.360936,0.182396,2.118974,NaN,0.108915,0.476132,0.285285,2.211227,0.000000,-2.812345,-0.000000,-0.470286,-0.465557,0.000000,NaN
BMO,0.352180,0.109392,3.590070,NaN,0.147335,0.629698,0.395515,1.346648,0.000000,0.345405,-0.505670,-0.190930,-0.212449,61.136109,NaN
AIG,-0.345883,-0.206481,1.758799,NaN,0.068714,0.199061,-0.187819,1.112205,0.000000,3.407498,1.395501,-4.616026,-4.635335,-96.915513,NaN
BK,0.307845,0.156908,2.098911,NaN,0.108792,0.609205,0.334107,1.176326,1.655940,1.655139,2.915119,-0.308596,-0.247366,-1375.227973,NaN


In [37]:
value_candidates = df[(df['ROE'] >= 0.08) & (df['Debt_Eq_Ratio'] <= 0.50) & (df['Current_Ratio'] >= 1.50) & (df['FCF_Growth'] > 0) & (df['Net_Income_Growth'] > 0) & (df['Operating_Margin'] > 0) & (df['Avg_Shares_Dil_Growth'] <= 0)]
value_candidates

,ROE,ROA,Debt_Eq_Ratio,Current_Ratio,PB,PS,PE,Dividend_Yield,Operating_Margin,Revenue_Growth,FCF_Growth,Net_Income_Growth,EPS_Growth,Avg_Shares_Dil_Growth,rank
TXN,2.381437,5.890509,0.251806,1.740461,2.356573,2.829402,0.935544,0.717927,2.430526,-0.154563,0.035422,0.191614,0.189688,-441.932946,490.287601
